# Analysez les ventes d'une entreprise de e-commerce

## Mise en situation

Vous êtes data analyst d'une grande chaîne de librairie, fraîchement embauché depuis une semaine !  
Il est temps de mettre les mains dans le cambouis ! Le service Informatique vous a donné l’accès à la base de données des ventes. À vous de vous familiariser avec les données, et de les analyser. Votre manager souhaite que vous réalisiez une présentation pour vous "faire la main".  
Comme vous l'avez appris dans vos recherches avant de postuler, votre entreprise, _"Rester livres"_ s'est d'abord développée dans une grande ville de France, avec plusieurs magasins, jusqu'à décider d'ouvrir une boutique en ligne. Son approche de la vente de livres en ligne, basée sur des **algorithmes de recommandation**, lui a valu un franc succès !

## Importation des modules

_Importation des modules nécessaires pour le nettoyage des données._

In [1]:
import pandas as pd
import numpy as np

## Chargement des données

_On procède au chargement du jeu de données composé de trois fichiers au format csv._

In [2]:
%pwd

'/Users/anissa'

In [3]:
%cd /Users/anissa/dataset_P4

/Users/anissa/dataset_P4


***csv customers***

In [4]:
customers = pd.read_csv("customers.csv")
customers.head()

client_id sex  birth
0    c_4410   f   1967
1    c_7839   f   1975
2    c_1699   f   1984
3    c_5961   f   1962
4    c_5320   m   1943

_Descriptif des colonnes_ :  
    client_id -> un id est donné à chaque client  
    sex -> genre des clients  
    birth -> année de naissance des clients  

***csv products***

In [5]:
products = pd.read_csv("products.csv")
products.head()

id_prod  price  categ
0  0_1421  19.99      0
1  0_1368   5.13      0
2   0_731  17.99      0
3   1_587   4.99      1
4  0_1507   3.99      0

_Descriptif des colonnes_ :  
    id_prod -> un id est attribué à chaque produit  
    price -> prix des produits  
    categ -> catégorie des produits  

***csv transactions***

In [6]:
transactions = pd.read_csv("transactions.csv")
transactions.head()

id_prod                        date session_id client_id
0  0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450
1   2_226  2022-02-03 01:55:53.276402   s_159142     c_277
2   1_374  2021-09-23 15:13:46.938559    s_94290    c_4270
3  0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597
4  0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242

_Descriptif des colonnes_ :  
    id_prod -> id attribué à chaque produit  
    date -> date à laquelle la transaction a été effectuée  
    session_id -> session pendant laquelle il y a eu un achat  
    client_id -> id de l'acheteur 

______________________________________________________________

# Mission n°1 - Nettoyage des données

### 1. Découverte des dataframes

#### 1. 1. Découverte du dataframe "customers"

In [7]:
# On détermine le nombre de clients (qui ont effectué un achat)
customers.shape

(8623, 3)

In [8]:
# On détermine la proportion clients du genre féminin
customers[customers["sex"]=="f"].shape

(4491, 3)

In [9]:
# On détermine la tranche d'année de naissance
customers["birth"].describe()

count    8623.000000
mean     1978.280877
std        16.919535
min      1929.000000
25%      1966.000000
50%      1979.000000
75%      1992.000000
max      2004.000000
Name: birth, dtype: float64

>> Il y a 8 623 clients, dont 4491 sont des femmes (soit 52% de femmes et 48% d'hommes). 
Leurs années de naissance s'étendent de 1929 à 2004.

#### 1. 2. Découverte du dataframe "products"

In [10]:
# On détermine le nombre de produits
products.shape

(3287, 3)

In [11]:
# On détermine le nombre de catégories de produits
products["categ"].unique()

array([0, 1, 2])

In [12]:
# Données sur les prix
products["price"].describe()

count    3287.000000
mean       21.856641
std        29.847908
min        -1.000000
25%         6.990000
50%        13.060000
75%        22.990000
max       300.000000
Name: price, dtype: float64

>> Il y a 3 287 produits dispersés en 3 catégories. 
Les prix s'étendent de -1€ à 300€. Le prix moyen est d'environ 22€.

#### 1. 3. Découverte du dataframe "transactions"

In [13]:
# On détermine le nombre de transactions
transactions.shape

(337016, 4)

In [14]:
# Données sur les transactions
transactions.describe()

id_prod                             date session_id client_id
count   337016                           337016     337016    337016
unique    3266                           336855     169195      8602
top      1_369  test_2021-03-01 02:30:02.237413        s_0    c_1609
freq      1081                               13        200     12855

>> 337 016 transactions ont été effectuées.

On constate ici qu'une ligne est présente 200 fois (session s_0)
* Hypothèse : cette session correspond à des tests

------

### 2. Nettoyage des données

Vérification de notre hypothèse, selon laquelle la session s_0 correspond à des tests.

In [15]:
# On observe plus en détails la session s_0 qui est suspecte
transactions.loc[transactions["session_id"]=="s_0"]

id_prod                             date session_id client_id
1431       T_0  test_2021-03-01 02:30:02.237420        s_0      ct_1
2365       T_0  test_2021-03-01 02:30:02.237446        s_0      ct_1
2895       T_0  test_2021-03-01 02:30:02.237414        s_0      ct_1
5955       T_0  test_2021-03-01 02:30:02.237441        s_0      ct_0
7283       T_0  test_2021-03-01 02:30:02.237434        s_0      ct_1
...        ...                              ...        ...       ...
332594     T_0  test_2021-03-01 02:30:02.237445        s_0      ct_0
332705     T_0  test_2021-03-01 02:30:02.237423        s_0      ct_1
332730     T_0  test_2021-03-01 02:30:02.237421        s_0      ct_1
333442     T_0  test_2021-03-01 02:30:02.237431        s_0      ct_1
335279     T_0  test_2021-03-01 02:30:02.237430        s_0      ct_0

[200 rows x 4 columns]

Les 200 lignes de la session s_0 correspondent aux clients ct_0 et ct_1, qui correspondent à l'id produit T_0, qui correspond lui-même à des dates commençant par "test". Il faut donc les supprimer.

#### 2. 1. Suppression des données suspectes

##### 2.1.1. Suppression du prix négatif

In [16]:
# On a constaté qu'il y avait un prix négatif "-1€". Il convient de le supprimer
produits = products.loc[products["price"]>=0]
produits.head()

id_prod  price  categ
0  0_1421  19.99      0
1  0_1368   5.13      0
2   0_731  17.99      0
3   1_587   4.99      1
4  0_1507   3.99      0

##### 2.1.2. Retrait des lignes "test" 

In [17]:
# Retrait des 200 lignes suspectes dans le df transactions
transacts = transactions.loc[transactions["id_prod"]!="T_0"]
transacts.head()

id_prod                        date session_id client_id
0  0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450
1   2_226  2022-02-03 01:55:53.276402   s_159142     c_277
2   1_374  2021-09-23 15:13:46.938559    s_94290    c_4270
3  0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597
4  0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242

In [18]:
# Retrait des 200 lignes suspectes dans le df customers
clients_clean = customers.loc[(customers["client_id"]!= "ct_0") 
                        & (customers["client_id"]!= "ct_1")]
clients_clean.head()

client_id sex  birth
0    c_4410   f   1967
1    c_7839   f   1975
2    c_1699   f   1984
3    c_5961   f   1962
4    c_5320   m   1943

##### 2.1.3. Travail sur les autres lignes nulles

In [19]:
# Jointure entre les df produits et transactions afin de vérifier s'il n'y a pas d'autres données suspectes
transacts_produits = pd.merge(transacts,produits, on="id_prod", how="left")
transacts_produits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 336815
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     336816 non-null  object 
 1   date        336816 non-null  object 
 2   session_id  336816 non-null  object 
 3   client_id   336816 non-null  object 
 4   price       336713 non-null  float64
 5   categ       336713 non-null  float64
dtypes: float64(2), object(4)
memory usage: 18.0+ MB


On remarque que les colonnes "price" et "categ" contiennent 103 lignes nulles. Il y a donc perte de données. On procède à son analyse afin de comprendre cette perte.

In [20]:
# Localisation des lignes nulles
lignes_nulles = transacts_produits[transacts_produits.isnull().any(axis=1)]
lignes_nulles.head()

id_prod                        date session_id client_id  price  categ
6231   0_2245  2021-06-17 03:03:12.668129    s_49705    c_1533    NaN    NaN
10797  0_2245  2021-06-16 05:53:01.627491    s_49323    c_7954    NaN    NaN
14045  0_2245  2021-11-24 17:35:59.911427   s_124474    c_5120    NaN    NaN
17480  0_2245  2022-02-28 18:08:49.875709   s_172304    c_4964    NaN    NaN
21071  0_2245  2021-03-01 00:09:29.301897        s_3     c_580    NaN    NaN

In [21]:
lignes_nulles.id_prod.unique()

array(['0_2245'], dtype=object)

On constate que cela vient uniquement du produit avec l'identifiant "0_2245". On vérifie alors si ce produit est bien présent dans le df produits et transactions.

In [22]:
# Vérification de la présence de l'id 0_2245
id_produits = transacts_produits.copy()
id_produits['id_prod_produits'] = id_produits['id_prod'].isin(produits['id_prod'])
id_produits['client_id_transactions'] = id_produits['client_id'].isin(transactions['client_id'])
id_produits.head()

id_prod                        date session_id client_id  price  categ  \
0  0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450   4.99    0.0   
1   2_226  2022-02-03 01:55:53.276402   s_159142     c_277  65.75    2.0   
2   1_374  2021-09-23 15:13:46.938559    s_94290    c_4270  10.71    1.0   
3  0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597   4.20    0.0   
4  0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242   8.99    0.0   

   id_prod_produits  client_id_transactions  
0              True                    True  
1              True                    True  
2              True                    True  
3              True                    True  
4              True                    True

In [23]:
# On affiche les données "False"
id_prod_false = id_produits[id_produits['id_prod_produits'] == False]
id_prod_false = id_prod_false.groupby('id_prod').mean()
print(id_prod_false)

         price  categ  id_prod_produits  client_id_transactions
id_prod                                                        
0_2245     NaN    NaN             False                    True


On constate ainsi que le produit "0_2245" n'est pas présent dans le df produits mais l'est dans le df transactions.  
On remarque que le produit 0_2245 n'a pas de référence dans le tableau produits (prix et catégorie) mais qu'il a été acheté 103 fois (transactions). On peut par conséquent déterminer les valeurs manquantes en l'estimant par la moyenne des valeurs (imputation par la moyenne).  
On peut supposer que le premier chiffre de chaque identifiant produit correspond à sa catégorie. On vérifie cette hypothèse, le produit 0_2245 appartient à la catégorie 0.

In [24]:
# Vérification de l'hypothèse 0 = catégorie 0
(produits["id_prod"].str[0].astype(int)==produits["categ"]).describe()

count     3286
unique       1
top       True
freq      3286
dtype: object

Hypothèse validée : le premier chiffre devant l'identifiant du produit correspond au numéro de la catégorie. On procède donc à l'imputation par la moyenne.

In [25]:
# Calcul du prix moyen de chaque catégorie
transacts_produits.groupby(by="categ").mean()

price
categ           
0.0    10.646828
1.0    20.480106
2.0    75.174949

In [26]:
# Imputation par la moyenne
transacts_produits["categ"] = transacts_produits[["categ"]].fillna(value=0)
transacts_produits["price"] = transacts_produits[["price"]].fillna(value=10.646828)
transacts_produits.info() # on constate qu'il n'y a plus de lignes nulles

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 336815
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     336816 non-null  object 
 1   date        336816 non-null  object 
 2   session_id  336816 non-null  object 
 3   client_id   336816 non-null  object 
 4   price       336816 non-null  float64
 5   categ       336816 non-null  float64
dtypes: float64(2), object(4)
memory usage: 18.0+ MB


----

### 3. Création de dataframes nettoyés

Une fois le nettoyage des données terminé, on procède à leur enregistrement afin de procéder par la suite à l'analyse des données. On crée 4 fichiers.

#### 3. 1. Création d'un dataframe unique

In [27]:
rester_livres = pd.merge(transacts_produits, clients_clean, 
                         on="client_id", how="left")
rester_livres.head()

#Ajout de la colonne âge
rester_livres["age"]= 2022 - rester_livres["birth"]

#Ajout de la colonne tranche_age
def tranche_age(age):
    tranches = [
        {'bornes': (0,18), 'label': '18-'},
        {'bornes': (19,30), 'label': '19-30'},
        {'bornes': (31,50), 'label': '31-50'},
        {'bornes': (51,70), 'label': '51-70'},
        {'bornes': (71,120),'label': '71+'}
    ]
    for t in tranches:
        if t['bornes'][0] <= age <= t['bornes'][1]:
            return t['label']
        
rester_livres['tranche_age'] = rester_livres['age'].apply(tranche_age)

# Ajout des colonnes pour les jours et moments de la journée
rester_livres['date'] = pd.to_datetime(rester_livres['date'])

rester_livres['jour'] = rester_livres['date'].map(lambda d: d.day)
rester_livres['mois'] = rester_livres['date'].map(lambda d: d.month)
rester_livres['heure'] = rester_livres['date'].map(lambda d: d.hour)

def moment_jour(value):
    if 14 <= value < 22:
        return 'Après-midi'
    elif 6 <= value < 14:
        return 'Matin'
    else:
        return 'Soir'

rester_livres['moment jour'] = rester_livres['heure'].map(moment_jour)

rester_livres['date'] = rester_livres['date'].dt.strftime('%Y %m %d')
rester_livres.head()

id_prod        date session_id client_id  price  categ sex  birth  age  \
0  0_1483  2021 04 10    s_18746    c_4450   4.99    0.0   f   1977   45   
1   2_226  2022 02 03   s_159142     c_277  65.75    2.0   f   2000   22   
2   1_374  2021 09 23    s_94290    c_4270  10.71    1.0   f   1979   43   
3  0_2186  2021 10 17   s_105936    c_4597   4.20    0.0   m   1963   59   
4  0_1351  2021 07 17    s_63642    c_1242   8.99    0.0   f   1980   42   

  tranche_age  jour  mois  heure moment jour  
0       31-50    10     4     18  Après-midi  
1       19-30     3     2      1        Soir  
2       31-50    23     9     15  Après-midi  
3       51-70    17    10      3        Soir  
4       31-50    17     7     20  Après-midi

In [28]:
# Enregistrement au format csv
rester_livres.to_csv("/Users/anissa/rester_livres.csv", index=False)

#### 3. 2. Création d'un dataframe pour les clients (ayant effectué un achat)

In [29]:
# Jointure entre le df clients_clients et rester_livres
clients_achat = pd.merge(clients_clean, rester_livres, 
                         how="inner",on="client_id")
clients_achat.head() # sauvegarde des données clients

client_id sex_x  birth_x id_prod        date session_id  price  categ sex_y  \
0    c_4410     f     1967  0_1455  2021 03 22     s_9942   8.99    0.0     f   
1    c_4410     f     1967  0_1376  2021 09 24    s_94984  16.24    0.0     f   
2    c_4410     f     1967   1_312  2022 01 29   s_156960  24.56    1.0     f   
3    c_4410     f     1967   1_653  2021 07 29    s_68860  25.99    1.0     f   
4    c_4410     f     1967  0_1110  2021 11 04   s_114715   4.71    0.0     f   

   birth_y  age tranche_age  jour  mois  heure moment jour  
0     1967   55       51-70    22     3     14  Après-midi  
1     1967   55       51-70    24     9     22        Soir  
2     1967   55       51-70    29     1     14  Après-midi  
3     1967   55       51-70    29     7     23        Soir  
4     1967   55       51-70     4    11     16  Après-midi

In [30]:
# Création du df clients
clients_list = ["client_id","sex_x","birth_x",
                "price", "categ","id_prod",
                "session_id","date"]

clients = clients_achat[clients_list] # données dont on a besoin

clients = clients.rename(columns={"sex_x":"sex","birth_x":"birth"})
clients.head()

client_id sex  birth  price  categ id_prod session_id        date
0    c_4410   f   1967   8.99    0.0  0_1455     s_9942  2021 03 22
1    c_4410   f   1967  16.24    0.0  0_1376    s_94984  2021 09 24
2    c_4410   f   1967  24.56    1.0   1_312   s_156960  2022 01 29
3    c_4410   f   1967  25.99    1.0   1_653    s_68860  2021 07 29
4    c_4410   f   1967   4.71    0.0  0_1110   s_114715  2021 11 04

In [31]:
# Enregistrement au format csv
clients.to_csv("/Users/anissa/clients.csv", index=False)

#### 3.3. Création d'un dataframe pour les visiteurs (n'ayant pas effectué d'achat)

In [32]:
# Jointure entre les différents df
df_visiteurs = pd.merge(left=products,right=transactions,
                        how="outer",on="id_prod")
df_visiteurs = pd.merge(left=customers,right=df_visiteurs, 
                        how="outer",on="client_id")

In [33]:
# On localise les données nulles de notre jointure
df_na = df_visiteurs[df_visiteurs.isnull().any(axis=1)]
df_na.head()

client_id sex   birth id_prod  price  categ                        date  \
2678    c_4505   m  1976.0  0_2245    NaN    NaN  2022-01-09 09:23:31.000720   
2968    c_3468   f  1981.0  0_2245    NaN    NaN  2021-09-11 10:52:05.205583   
6683    c_1403   f  1978.0  0_2245    NaN    NaN  2022-02-15 14:26:50.187952   
8386    c_3065   f  1977.0  0_2245    NaN    NaN  2022-01-26 13:34:33.440366   
9457    c_7102   m  1983.0  0_2245    NaN    NaN  2021-04-25 19:58:42.716401   

     session_id  
2678   s_147220  
2968    s_88251  
6683   s_165575  
8386   s_155484  
9457    s_25704

In [34]:
# On localise les id des visiteurs
visiteurs_sans_achat = df_na.loc[df_visiteurs.id_prod.isnull(),
                                         "client_id"].unique()
visiteurs_sans_achat

array(['c_8253', 'c_3789', 'c_4406', 'c_2706', 'c_3443', 'c_4447',
       'c_3017', 'c_4086', 'c_6930', 'c_4358', 'c_8381', 'c_1223',
       'c_6862', 'c_5245', 'c_5223', 'c_6735', 'c_862', 'c_7584', 'c_90',
       'c_587', 'c_3526'], dtype=object)

In [35]:
# On crée le df contenant les visiteurs n'ayant pas effectué d'achat
visiteurs = df_visiteurs.loc[df_visiteurs["client_id"]
                            .isin(visiteurs_sans_achat),
                             ["client_id","sex","birth"]]
visiteurs.head()

client_id sex   birth
29516     c_8253   f  2001.0
99355     c_3789   f  1997.0
108189    c_4406   f  1998.0
109402    c_2706   f  1967.0
112024    c_3443   m  1959.0

In [36]:
# Enregistrement au format csv
visiteurs.to_csv("/Users/anissa/visiteurs.csv", index=False)

#### 3. 4. Création d'un dataframe pour les produits invendus

In [37]:
# On localise les données nulles de notre df
p_invendus = df_na.loc[df_visiteurs.client_id.isnull(),"id_prod"].unique()
p_invendus

array(['0_1016', '0_1780', '0_1062', '0_1119', '0_1014', '1_0', '0_1318',
       '0_1800', '0_1645', '0_322', '0_1620', '0_1025', '2_87', '1_394',
       '2_72', '0_310', '0_1624', '0_525', '2_86', '0_299', '0_510',
       '0_2308'], dtype=object)

In [38]:
# On crée le df des produits invendus
produits_invendus = df_visiteurs.loc[df_visiteurs["id_prod"]
                                     .isin(p_invendus), 
                                     ["id_prod","categ","price"]]
produits_invendus.head()

id_prod  categ  price
337037  0_1016    0.0  35.06
337038  0_1780    0.0   1.67
337039  0_1062    0.0  20.08
337040  0_1119    0.0   2.99
337041  0_1014    0.0   1.15

In [39]:
# Enregistrement au format csv
produits_invendus.to_csv("/Users/anissa/produits_invendus.csv", index=False)

### 4. Enregistrement des données nettoyées

In [40]:
produits.to_csv("/Users/anissa/produits.csv", index=False)
transacts.to_csv("/Users/anissa/transacts.csv", index=False)
clients_clean.to_csv("/Users/anissa/clients_clean.csv", index=False)
transacts_produits.to_csv("/Users/anissa/transacts_produits.csv", index=False)